# Association of the mitochondria to the neuronal skeletons


In [1]:
# load a cell, its skeleton and the mito dataframe
import pandas 

cell_df = pandas.read_csv("smo_data/inh_cell.csv", header=0, names=["cellid"])
example_cell = cell_df.iloc[0]

# for each mito, load the mesh and get the closest node for all mesh nodes

# two alternatives navis.snap or via KDtree and the mesh skeleton

# combine the found nodes into a unique set

# write to file

In [2]:
import tqdm

import navis
import os
import sys
sys.path.append(".")
from lib import mesh


def get_cell_with_synapses_and_mitodfs(cellid):
    cell_df = pandas.read_csv("smo_data/cell_type_classification_filtered.csv", index_col=0)
    neuron = navis.read_swc("smo_data/cell_morphologies/{:d}.swc".format(cellid))
    synapse_df = pandas.read_csv("smo_data/synapse_filtered.csv", index_col=0).query("cell_id==@cellid")
    mito_df = pandas.read_csv(
        "smo_data/microns_cells/{:s}_{:d}.csv".format(
        cell_df.loc[cellid].cell_type, cellid), index_col=0
    )
    return neuron, synapse_df, mito_df



def get_associated_nodes(mito_sr, neuron):
    mitoid = mito_sr["mito_id"]
    mito = mesh.read_mito_mesh(mitoid)

    assoc_nodes, dist = neuron.snap(mito.vertices)
    assoc_nodes = list(set(assoc_nodes))
    return assoc_nodes


def get_all_associated_nodes(neuron, mito_df):
    assoc_nodes_df = mito_df.set_index("mito_id", drop=False).apply(
        lambda sr: get_associated_nodes(sr, neuron), axis=1)
    
    return assoc_nodes_df

def write_mitochondria_and_associated_nodes(cellid, path):
    neuron, synapse_df, mito_df = get_cell_with_synapses_and_mitodfs(cellid)
    assoc_nodes_df = get_all_associated_nodes(neuron, mito_df.query("compartment!='Somatic'"))
    filename = os.path.join(path, "{:d}_assoc_nodes.json".format(cellid))
    assoc_nodes_df.to_json(filename)
    
def write_all_mitochondria_and_associated_nodes(cell_df, path):
    for idx, cell_sr in tqdm.tqdm(cell_df.iterrows(), total=len(cell_df), position=0, leave=True):
        cellid = int(cell_sr.name)
        try:
            write_mitochondria_and_associated_nodes(cellid, "smo_data/associated_nodes/")
        except BaseException as e:
            print(e)
            tqdm.tqdm.write("Problems with cell {:d}".format(cellid))

    

In [3]:
cell_df = pandas.read_csv("smo_data/cell_type_classification_filtered.csv", index_col=0)

In [4]:
!mkdir -p smo_data/associated_nodes

In [5]:
write_all_mitochondria_and_associated_nodes(cell_df, "smo_data/associated_nodes/")

100%|████████████████████| 384/384 [07:30<00:00,  1.17s/it]
